# Sw00pGenerator3000

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import peakutils as pu
from plotly.subplots import make_subplots

import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from math import sqrt, degrees, atan
import geopy.distance
import pyproj
import utm

from jupyter_dash import JupyterDash 
from dash import Dash, dcc, html, Input, Output, callback

import logging
logging.basicConfig(level=logging.INFO)

themes = ["plotly_white", "plotly_dark", "ggplot2", "simple_white"]
pio.templates.default = themes[0]

In [2]:
token = "pk.eyJ1IjoiYWlieWJydW0iLCJhIjoiY2xhdmRldnhyMDRkNTNybWg0NTl6eHgwZSJ9.phqQCg9UWIXx5wFkc_i0kg"

raw_df = pd.read_csv('../data/raw_own/23-04-09/J1.CSV', skiprows=[1])
raw_df.head()

,time,lat,lon,hMSL,velN,velE,velD,hAcc,vAcc,sAcc,heading,cAcc,gpsFix,numSV
0,2023-04-09T07:28:07.40Z,51.024551,5.520155,1348.562,-16.88,5.83,21.29,4307.536,5007.471,224.70,160.95330,7.66592,3,4
1,2023-04-09T07:28:07.60Z,51.023893,5.520543,1260.528,-18.41,6.39,23.06,1904.139,2209.380,96.17,160.86574,3.90365,3,4
2,2023-04-09T07:28:07.80Z,51.016739,5.524668,283.871,-1.86,0.36,2.70,1174.856,1382.874,39.04,160.86574,4.71886,3,4
3,2023-04-09T07:28:08.00Z,51.015957,5.525116,174.914,0.29,-0.04,-0.66,803.097,952.791,23.55,160.86574,5.14861,3,4
4,2023-04-09T07:28:08.20Z,51.015678,5.525263,134.521,0.00,-0.01,-0.04,607.534,727.155,14.30,160.86574,5.39289,3,5


In [3]:
def validate_metric(speed_metric, distance_metric):
    if speed_metric != 'km/u' and speed_metric != 'mph':
        raise ValueError("Wrong speed metric")
    if distance_metric != 'm' and distance_metric != 'ft':
        raise ValueError("Wrong distance metric")


def get_y_axis_settings(df, speed_metric='km/u', distance_metric='m'):
    validate_metric(speed_metric, distance_metric)
    return {
        'Elevation':
        {
            'col': df.elevation, 
            'color': '#636EFA', 
            'metric': "ft",
            'hovertemplate': 'Elevation: %{y:.2f} ft <extra></extra>'
        },
        'Horizontal speed': 
        {
            'col': df['horz_speed_km/u'] if speed_metric == 'km/u' else df['horz_speed_mph'], 
            'color': '#FF0B0B', 
            'metric': speed_metric,
            'hovertemplate': 'Horz speed: %{y:.2f} ' + speed_metric + '<extra></extra>'
        },
        'Vertical speed':   
        {
            'col': df['vert_speed_km/u'] if speed_metric == 'km/u' else df['vert_speed_mph'],
            'color': '#00CC96', 
            'metric': speed_metric,
            'hovertemplate': 'Vert speed: %{y:.2f} ' + speed_metric + '<extra></extra>'
        },
        'Dive angle':       
        {
            'col': df.dive_angle,
            'color': '#AB63FA', 
            'metric': 'deg',
            'hovertemplate': 'Dive angle: %{y:.2f}° <extra></extra>'
        },
        'Distance':   
        {
            'col': df['y_axis_distance_m'] if distance_metric == "m" else df['y_axis_distance_ft'],
            'color': '#636EFA', 
            'metric': distance_metric,
            'hovertemplate': 'yaxis distance: %{y:.2f} ' + distance_metric + '<extra></extra>'
        },
    }

def get_x_axis_settings(df, speed_metric='km/u', distance_metric='m'):
    validate_metric(speed_metric, distance_metric)
    return {
        'Time': 
        {
            'col': df.time,
            'metric': 's'
        },
        'Horizontal Distance': 
        {
            'col': df['horz_distance_m'] if distance_metric == "m" else df['horz_distance_ft'],
            'metric': distance_metric
        },
        'Distance': 
        {
            'col': df['x_axis_distance_m'] if distance_metric == "m" else df['x_axis_distance_ft'],
            'metric': distance_metric
        }
    }

def get_metrics(df, idke, speed_metric):
    y_axis = get_yaxis_settings(df, speed_metric)
    return {
        'y_axis':
        {
            'values': [y_axis[col]['col'].loc[y_axis[col]['col'].index == idke].values[0] for col in y_axis.keys()], 
            'metric': [y_axis[col]['metric'] for col in y_axis.keys()],
        },
        
    }

In [4]:
# Dataset helpers
def meters_to_feet(meters):
    return meters * 3.280839895


def feet_to_meters(feet):
    return feet / 3.280839895


def meter_per_second_to_miles_per_hour(meter_per_second):
    return meter_per_second * 2.236936


def meter_per_second_to_kilometers_per_hour(meter_per_second):
    return meter_per_second * 3.6


def calc_horizontal_speed(n, e):
    return sqrt((n**2) + (e**2))


def calc_dive_angle(v_speed, h_speed):
    try:
        return degrees(atan(v_speed/h_speed))
    except ZeroDivisionError:
        return 0


def calc_distance_geo(metric, lat1, lat2, lon1, lon2):
    coordinates_1 = (lat1, lon1)
    coordinates_2 = (lat2, lon2)
    if metric == 'ft':
        return geopy.distance.geodesic(coordinates_1, coordinates_2).miles * 5280
    elif metric == 'm':
        return geopy.distance.geodesic(coordinates_1, coordinates_2).kilometers * 1000


def to_utm(lat, lon):
    zone = utm.from_latlon(lat[0], lon[0])[2]
    p = pyproj.Proj(proj='utm', zone=zone, ellps='WGS84')
    return p(lon, lat)


def calc_axis_distance(metric, lat, lon):
    x, y = to_utm(lat, lon)
    if metric == 'ft':
        return [meters_to_feet(x_cor) for x_cor in x], [meters_to_feet(y_cor) for y_cor in y]
    elif metric == 'm':
        return x, y

In [5]:
# Creating dataset

class Dataset:
    def __init__(self, filename, df):
        self.filename = filename
        self.df = df

    def get_total_seconds(self):
        datetimes = [pd.to_datetime(d) for d in self.df.time]
        l = []
        for i, d in enumerate(datetimes):
            duration = datetimes[i] - datetimes[0]
            l.append(duration.total_seconds())
        return l

    def get_fixed_elevation(self, elevation):
        ground_elevation = meters_to_feet(self.df.hMSL.iloc[-1])
        return [meters_to_feet(self.df.hMSL[i]) - ground_elevation - elevation for i in
                range(0, len(self.df.hMSL))]
        # return [helpers.meters_to_feet(self.df.hMSL[i]) - elevation for i in range(0, len(self.df.hMSL))]

    def get_vertical_speed(self, metric):
        if metric == 'mph':
            return [meter_per_second_to_miles_per_hour(meter) for meter in self.df.velD]
        elif metric == 'km/u':
            return [meter_per_second_to_kilometers_per_hour(meter) for meter in self.df.velD]
        else:
            raise ValueError('Invalid metric')

    def get_horizontal_speed(self, metric):
        speed = [calc_horizontal_speed(self.df.velN[i], self.df.velE[i]) for i in range(0, len(self.df))]
        if metric == 'mph':
            return [meter_per_second_to_miles_per_hour(s) for s in speed]
        elif metric == 'km/u':
            return [meter_per_second_to_kilometers_per_hour(s) for s in speed]
        else:
            raise ValueError('Invalid metric')

    def get_dive_angle(self, v_speed, h_speed):
        return [calc_dive_angle(v_speed[i], h_speed[i]) for i in range(0, len(self.df))]

    def get_horizontal_distance(self, metric):
        lis, dis = [0], 0.0
        for i in range(0, len(self.df) - 1):
            dis += calc_distance_geo(metric, self.df.lat[i], self.df.lat[i + 1], self.df.lon[i],
                                             self.df.lon[i + 1])
            lis.append(dis)
        return lis

    def get_axis_distance(self, metric, axis):
        x, y = calc_axis_distance(metric, self.df.lat, self.df.lon)
        axis_value = x if axis == 'x' else y
        lis, dis = [0], 0.0
        for i in range(0, len(axis_value) - 1):
            dis += axis_value[i + 1] - axis_value[i]
            lis.append(dis)
        return lis

    def create(self):
        df = pd.DataFrame({
            'timestamp': pd.to_datetime(self.df.time),
            'time': np.array(self.get_total_seconds()),
            'lat': self.df.lat,
            'lon': self.df.lon,
            'elevation': self.get_fixed_elevation(0),
            'horz_distance_ft': self.get_horizontal_distance('ft'),
            'horz_distance_m': self.get_horizontal_distance('m'),
            'x_axis_distance_ft': self.get_axis_distance('ft', 'x'),
            'x_axis_distance_m': self.get_axis_distance('m', 'x'),
            'y_axis_distance_ft': self.get_axis_distance('ft', 'y'),
            'y_axis_distance_m': self.get_axis_distance('m', 'y'),
            'vert_speed_mph': self.get_vertical_speed('mph'),
            'horz_speed_mph': self.get_horizontal_speed('mph'),
            'vert_speed_km/u': self.get_vertical_speed('km/u'),
            'horz_speed_km/u': self.get_horizontal_speed('km/u'),
            'heading': self.df.heading,
            'dive_angle': self.get_dive_angle(self.get_vertical_speed('mph'), self.get_horizontal_speed('mph')),
            'user': 0})
        #df.set_index('timestamp', inplace=True)
        return df

    def get_name(self):
        return pd.to_datetime(self.df.time[0]).strftime("D%m-%d-%YT%H%M") + self.filename

    def save(self, path):
        dataframe = self.create()
        dataframe.to_csv(os.path.join(path, self.get_name() + '.csv'), index=False)

    def __str__(self):
        return f'{self.get_name()}'

raw = pd.read_csv('../data/raw_own/23-04-15/J3.CSV', skiprows=[1])

#raw = pd.read_csv('../data/raw_own/23-04-09/12-26-52.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-08/12-10-38.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-08/15-27-08.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-08/15-29-54.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-08/17-17-49.CSV', skiprows=[1])


#raw = pd.read_csv('../data/raw/J2.CSV', skiprows=[1])
dataset = Dataset("v1", raw).create()
dataset.head()

,timestamp,time,lat,lon,elevation,horz_distance_ft,horz_distance_m,x_axis_distance_ft,x_axis_distance_m,y_axis_distance_ft,y_axis_distance_m,vert_speed_mph,horz_speed_mph,vert_speed_km/u,horz_speed_km/u,heading,dive_angle,user
0,2023-04-15 11:03:47.200000+00:00,0.0,50.849336,3.144861,209.317585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.762168,1.361778,12.492,2.191569,0.00000,80.049416,0
1,2023-04-15 11:03:47.400000+00:00,0.2,50.848892,3.144476,-6.725722,184.927293,56.365839,-88.741362,-27.048367,-162.159824,-49.426314,-1.051360,2.381680,-1.692,3.832944,0.00000,-23.818399,0
2,2023-04-15 11:03:47.600000+00:00,0.4,50.848911,3.144473,8.585958,192.211499,58.586065,-89.309868,-27.221648,-154.900749,-47.213748,0.223694,2.721074,0.360,4.379145,0.00000,4.699594,0
3,2023-04-15 11:03:47.800000+00:00,0.6,50.848913,3.144479,8.431759,193.601181,59.009640,-88.087054,-26.848934,-154.241658,-47.012857,-0.984252,2.784334,-1.584,4.480952,0.00000,-19.468315,0
4,2023-04-15 11:03:48+00:00,0.8,50.848921,3.144480,9.730971,196.495690,59.891886,-87.838632,-26.773215,-151.358988,-46.134220,0.246063,3.768490,0.396,6.064798,298.86286,3.735815,0


In [6]:
def closest_value(input_list, input_value):
    difference = lambda input_list : abs(input_list - input_value)
    res = min(input_list, key=difference)
    return res

class Exit:
    def __init__(self, df):
        self.df = df
        self.exit_df = self.df.iloc[self.get_exit():].reset_index(drop=True)

    def get_exit_df(self):
        return self.exit_df

    def get_skydive_elevation(self):
        peaks = pu.indexes(self.df.elevation, thres=0.9, min_dist=1)
        # offset because the exit point can be to late to measure horz, vert speed
        offset = peaks[-1] - 20
        jmp_df = self.df.iloc[offset:].reset_index()
        return {'plane_on_altitude': jmp_df, 'peaks': peaks}

    def get_skydive_horz_speed(self):
        jmp_df = self.get_skydive_elevation()['plane_on_altitude']
        # Get horz speed before exit / plane on attitude
        horz_speed_peaks = pu.indexes(jmp_df.horz_speed_mph, thres=0.5, min_dist=1)
        horz_speed_lows = pu.indexes(-jmp_df.horz_speed_mph, thres=0.5, min_dist=1)
        peak = [p for p in horz_speed_peaks if p < horz_speed_lows[0]]
        return {'plane_on_altitude': jmp_df, 'exit': peak[-1], 'peaks': horz_speed_peaks, 'lows': horz_speed_lows}

    def get_skydive_vert_speed(self):
        jmp_df = self.get_skydive_elevation()['plane_on_altitude']
        # Get horz speed before exit / plane on attitude
        vert_speed_peaks = pu.indexes(jmp_df.vert_speed_mph, thres=0.5, min_dist=1)
        vert_speed_lows = pu.indexes(-jmp_df.vert_speed_mph, thres=0.5, min_dist=1)
        drop = [l for l in vert_speed_lows if l < vert_speed_peaks[0]]
        return {'plane_on_altitude': jmp_df, 'exit': drop[-1], 'peaks': vert_speed_peaks, 'lows': vert_speed_lows}

    def get_exit(self):
        jmp_df = self.get_skydive_elevation()['plane_on_altitude']
        mean = np.mean([jmp_df.time[self.get_skydive_vert_speed()['exit']], jmp_df.time[self.get_skydive_horz_speed()['exit']]])
        return self.df.index[self.df['time'] == closest_value(self.df.time, mean)][0]

    def plt_exit_point(self):
        jmp_df = self.get_skydive_elevation()['plane_on_altitude']

        gs = gridspec.GridSpec(2, 2)
        pl.figure(figsize=(15,10))

        ax = pl.subplot(gs[1, 0]) # row 0, col 0
        horz_speed = self.get_skydive_horz_speed()

        pl.plot(jmp_df.time, jmp_df.horz_speed_mph)
        pl.axvline(x = jmp_df.time[horz_speed['exit']], color='grey', linestyle ="--")
        pl.plot(jmp_df.time[horz_speed['peaks']], jmp_df.horz_speed_mph[horz_speed['peaks']], marker="8", color='g', ls="")
        pl.plot(jmp_df.time[horz_speed['lows'][0]], jmp_df.horz_speed_mph[horz_speed['lows'][0]], marker="8", color='r', ls="")
        pl.title("exit - Horizontal speed")
        pl.xlabel("Time (s)")
        pl.ylabel("Horizontal speed (mph)")

        ax = pl.subplot(gs[1, 1]) # row 0, col 1
        vert_speed = self.get_skydive_vert_speed()

        pl.plot(jmp_df.time, jmp_df.vert_speed_mph)
        pl.axvline(x = jmp_df.time[vert_speed['exit']], color='grey', linestyle ="--")
        pl.plot(jmp_df.time[vert_speed['peaks'][0]], jmp_df.vert_speed_mph[vert_speed['peaks'][0]], marker="8", color='g', ls="")
        pl.plot(jmp_df.time[vert_speed['lows']], jmp_df.vert_speed_mph[vert_speed['lows']], marker="8", color='r', ls="")
        pl.title("exit - Vertical speed")
        pl.xlabel("Time (s)")
        pl.ylabel("Vertical speed (mph)")

        ax = pl.subplot(gs[0, :]) # row 1, span all columns
        peaks = self.get_skydive_elevation()['peaks']

        pl.plot(self.df.time, self.df.elevation)
        pl.axvline(x = self.df.time[peaks[-1]], color='grey', linestyle ="--")
        pl.plot(self.df.time[peaks], self.df.elevation[peaks], marker="8", color='g', ls="")
        pl.title("Plane on altitude - Elevation")
        pl.xlabel("Time (s)")
        pl.ylabel("Elevation (feet)")

        pl.show()

    def plt_exit(self):
        fig, ax = plt.subplots(figsize=(15,8))
        fig.subplots_adjust(right=0.75)

        twin1 = ax.twinx()
        twin2 = ax.twinx()

        twin2.spines.right.set_position(("axes", 1.08))

        p1, = ax.plot(self.exit_df.horz_distance_m, self.exit_df.elevation, "b", label="Elevation")
        p2, = twin1.plot(self.exit_df.horz_distance_m, self.exit_df.horz_speed_mph, "r", label="Horz speed")
        p3, = twin2.plot(self.exit_df.horz_distance_m, self.exit_df.vert_speed_mph, "g", label="Vert speed")

        ax.set_xlabel("Distance (feet)")
        ax.set_ylabel("Elevation (feet)")
        twin1.set_ylabel("Horizontal speed (mph)")
        twin2.set_ylabel("Vertical speed (mph)")

        ax.yaxis.label.set_color(p1.get_color())
        twin1.yaxis.label.set_color(p2.get_color())
        twin2.yaxis.label.set_color(p3.get_color())

        tkw = dict(size=4, width=1.5)
        ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
        twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
        twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
        ax.tick_params(axis='x', **tkw)

        ax.legend(handles=[p1, p2, p3])
        plt.show()

In [7]:
class Landing:
    def __init__(self, df):
        self.df = df
        self.exit_df = Exit(df).get_exit_df()
        self.landing_df = self.exit_df.iloc[self.get_landing_index():].reset_index(drop=True)
    
    def get_landing_df(self):
        return self.landing_df
        
    # Get landing
    def get_landing_elevation(self):
        jmp_df = self.exit_df[self.exit_df.elevation <= 1200].reset_index()
        peaks = pu.indexes(jmp_df.elevation, thres=0.5, min_dist=0.1)
        lows = pu.indexes(-jmp_df.elevation, thres=0.1, min_dist=0.1)
        return {'start_landing': jmp_df, 'peaks': peaks, 'lows': lows}

    def get_landing_vert_speed(self):
        jmp_df = self.get_landing_elevation()['start_landing']
        peaks = pu.indexes(jmp_df.vert_speed_mph, thres=0.5, min_dist=0.1)
        lows = pu.indexes(-jmp_df.vert_speed_mph, thres=0.75, min_dist=5)
        
        lows_keys = [low for low in lows if low <= peaks[-1]]
        lows_values = [jmp_df.vert_speed_mph[low] for low in lows_keys]
        
        # landing_point is where the vertspeed is the lowest and start to increase
        return {'landing_point': lows_keys[lows_values.index(min(lows_values))], 'peaks': peaks, 'lows': lows}
    
    def get_landing(self):
        # change landing point if there is a dip in elevation
        elevation_low = self.get_landing_elevation()['lows'][0]
        vert_speed_low = self.get_landing_vert_speed()['landing_point']
        if elevation_low <= vert_speed_low:
            return elevation_low
        return vert_speed_low
    
    def get_landing_index(self):
        try:
            jmp_df = self.get_landing_elevation()['start_landing']
            return self.exit_df.index[self.exit_df['time'] == jmp_df.time[self.get_landing()]][0]
        except:
            logging.info(f'No high performance landing detected')
        
    # Get features of landing
    def get_stop(self):
        offset = 1.5
        horz_speed_peaks = pu.indexes(self.landing_df.horz_speed_mph, thres=0.1, min_dist=1)
        horz_speed_lows = pu.indexes(-self.landing_df.horz_speed_mph, thres=0.5, min_dist=1)
        return [l for l in horz_speed_lows if l > horz_speed_peaks[-1] and self.landing_df.horz_speed_mph[l] < offset][0]
    
    def get_top_of_turn(self):
        peaks = pu.indexes(self.landing_df.elevation, thres=0.01, min_dist=1)
        if len(peaks) == 0 or peaks[0] >= self.get_stop():
            return None
        return peaks[0]

    def get_max_horz_speed(self):
        return self.landing_df.idxmax().horz_speed_mph
        
    # Visualisations
    def plt_landing_point(self):
        jmp_df = self.get_landing_elevation()['start_landing']
        elevation = self.get_landing_elevation()
        vert_speed = self.get_landing_vert_speed()
        
        gs = gridspec.GridSpec(2, 1)
        pl.figure(figsize=(15,10))
        
        ax = pl.subplot(gs[0, 0]) # row 1, span all columns
        
        pl.plot(jmp_df.time, jmp_df.elevation)
        pl.axvline(x = jmp_df.time[vert_speed['landing_point']], color='grey', linestyle ="--")
        pl.plot(jmp_df.time[elevation['peaks']], jmp_df.elevation[elevation['peaks']], marker="8", color='g', ls="")
        pl.plot(jmp_df.time[elevation['lows']], jmp_df.elevation[elevation['lows']], marker="8", color='r', ls="")
        pl.title("Landing - Elevation")
        pl.xlabel("Time (s)")
        pl.ylabel("Elevation (feet)")

        ax = pl.subplot(gs[1, 0]) # row 0, col 1

        pl.plot(jmp_df.time, jmp_df.vert_speed_mph)
        pl.axvline(x = jmp_df.time[vert_speed['landing_point']], color='grey', linestyle ="--")
        pl.plot(jmp_df.time[vert_speed['peaks']], jmp_df.vert_speed_mph[vert_speed['peaks']], marker="8", color='g', ls="")
        pl.plot(jmp_df.time[vert_speed['lows']], jmp_df.vert_speed_mph[vert_speed['lows']], marker="8", color='r', ls="")
        pl.title("Landing - Vertical speed")
        pl.xlabel("Time (s)")
        pl.ylabel("Vertical speed (mph)")

        pl.show() 
        
    def plt_side_view(self, distance_metric):
        x_axis = get_x_axis_settings(self.landing_df, distance_metric=distance_metric)
        y_axis = get_y_axis_settings(self.landing_df)
        
        top = self.get_top_of_turn()

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=x_axis['Horizontal Distance']['col'].iloc[:self.get_stop()+1],
            y=y_axis['Elevation']['col'].iloc[:self.get_stop()+1],
            name='Elevation',
            line=dict(color=y_axis['Elevation']['color'], width=1.2),
            hovertemplate=y_axis['Elevation']['hovertemplate'],
            showlegend=False
        ))
        if top is not None:
            fig.add_trace(go.Scatter(
                x=[x_axis['Horizontal Distance']['col'][top]],
                y=[y_axis['Elevation']['col'][top]],
                name='Top of turn',
                text=['Top of turn'],
                textposition='top right',
                hovertemplate='Top of turn: %{y:.2f}' + y_axis['Elevation']['metric'] + ' <extra></extra>',
                mode='markers+text',
                showlegend=False
            ))
        fig.add_trace(go.Scatter(
            x=[x_axis['Horizontal Distance']['col'][self.get_stop()]],
            y=[y_axis['Elevation']['col'][self.get_stop()]],
            name='Stop',
            text=['stop'],
            textposition='top left',
            hovertemplate='Stop: %{y:.2f}' + x_axis['Distance']['metric'] + ' <extra></extra>',
            mode='markers+text',
            showlegend=False
        ))
        fig.update_layout(hovermode='x unified',
                          legend=dict(yanchor="top", y=1, xanchor="right", x=1))
        fig.update_xaxes(title=f"Horizontal distance ({x_axis['Horizontal Distance']['metric']})")
        fig.update_yaxes(title=f"Elevation ({y_axis['Elevation']['metric']})")

        return fig
    
    def plt_speed(self, speed_metric, distance_metric):
        x_axis = get_x_axis_settings(self.landing_df, distance_metric=distance_metric)
        y_axis = get_y_axis_settings(self.landing_df, speed_metric=speed_metric)

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=x_axis['Horizontal Distance']['col'].iloc[self.get_max_horz_speed():self.get_stop() + 1],
            y=y_axis['Horizontal speed']['col'].iloc[self.get_max_horz_speed():self.get_stop() + 1],
            line=dict(color=y_axis['Horizontal speed']['color'], width=1.2),
            hovertemplate=y_axis['Horizontal speed']['hovertemplate'],
            showlegend=False
        ))
        fig.add_trace(go.Scatter(
            x=[x_axis['Horizontal Distance']['col'].iloc[self.get_max_horz_speed()]],
            y=[y_axis['Horizontal speed']['col'][self.get_max_horz_speed()]],
            name=f"max horz speed ({round(y_axis['Horizontal speed']['col'][self.get_max_horz_speed()], 2)} {y_axis['Horizontal speed']['metric']})",
            text=["Max Horz Speed"],
            textposition="top right",
            hovertemplate='max horz speed: %{y:.2f}' + y_axis['Horizontal speed']['metric'] + ' <extra></extra>',
            mode='markers+text'
        ))
        fig.add_trace(go.Scatter(
            x=[x_axis['Horizontal Distance']['col'].iloc[self.get_stop()]],
            y=[y_axis['Horizontal speed']['col'][self.get_stop()]],
            text=['stop'],
            textposition='top left',
            hovertemplate='Stop: %{y:.2f}' + x_axis['Horizontal Distance']['metric'] + ' <extra></extra>',
            mode='markers+text',
            showlegend=False
        ))

        fig.update_layout(hovermode='x unified',
                          legend=dict(yanchor="top", y=1, xanchor="right", x=1))
        fig.update_xaxes(title=f"Horizontal distance ({x_axis['Horizontal Distance']['metric']})")
        fig.update_yaxes(title=f"Horizontal speed ({y_axis['Horizontal speed']['metric']})")
        return fig

In [8]:
class Jump(Landing, Exit):
    def __init__(self, name="empty", df=pd.DataFrame()):
        self.name = name
        if not df.empty:
            self.df = df
            super().__init__(self.df)

    def get_df(self):
        return self.df

    def get_name(self):
        return self.name

    def set_name(self, name):
        self.name = name

    def __str__(self):
        return f'{self.name}'

In [9]:
jump = Jump(name="test", df=dataset)

IndexError: list index out of range

In [16]:
jump.plt_exit_point()

NameError: name 'jump' is not defined

In [ ]:
jump.plt_landing_point()

In [ ]:
jump.plt_side_view('ft')

In [188]:
# OLD Code

#def get_difference(df, fase, between=None):
#    if fase == '1':
#        lis, diff = [0], 0.0
#    if fase == '2':
#        lis, diff = [round(between, 6)], between
#    for i in range(0, len(df)-1):
#        diff += df[i+1] - df[i]
#        lis.append(round(diff, 6))
#    return lis

#def shift_df(dataframe, key, col):
#    first = dataframe.iloc[:key+1][::-1][col].reset_index(drop=True)
#    second = dataframe.iloc[key+1:][col].reset_index(drop=True)
#    if key < 0:
#        raise ValueError("key cannot be equal or smaller then 0")
#    if key > len(dataframe):
#        raise ValueError("key cannot be bigger then length of dataframe")
#    if key == len(dataframe):
#        return get_difference(first, '1')[::-1]
#    between = second.iloc[0] - first.iloc[0]
#    return np.concatenate((get_difference(first, '1')[::-1], get_difference(second, '2', between)))

In [182]:
def cumulative_sum(df, fase='1', between=0.0):
    lis, diff = [0.0], 0.0
    if fase == '2':
        lis[0] = round(between, 6)
        diff = lis[0]
    for i in range(len(df) - 1):
        diff += df[i+1] - df[i]
        lis.append(round(diff, 6))
    return lis


def shift_df(dataframe, key, col):
    if key < 0:
        raise ValueError("key must be positive")
    if key >= len(dataframe):
        raise ValueError("key must be less than the length of dataframe")
    first = dataframe.iloc[:key+1][col][::-1].reset_index(drop=True)
    second = dataframe.iloc[key+1:][col].reset_index(drop=True)
    between = second.iloc[0] - first.iloc[0]
    return np.concatenate((cumulative_sum(first, fase='1')[::-1], cumulative_sum(second, fase='2', between=between)))


def set_start_point(df, key):
    df2 = df.copy()
    df2['time'] = shift_df(df2, key, 'time')
    df2['horz_distance_m'] = shift_df(df2, key, 'horz_distance_m')
    df2['horz_distance_ft'] = shift_df(df2, key, 'horz_distance_ft')
    df2['x_axis_distance_m'] = shift_df(df2, key, 'x_axis_distance_m')
    df2['x_axis_distance_ft'] = shift_df(df2, key, 'x_axis_distance_ft')
    df2['y_axis_distance_m'] = shift_df(df2, key, 'y_axis_distance_m')
    df2['y_axis_distance_ft'] = shift_df(df2, key, 'y_axis_distance_ft')
    return df2

In [30]:
test_df = set_start_point(landing_df, get_top_of_turn())
# test_df = set_start_point(landing_df, 0)
plt_side_view(test_df, 'm')

In [35]:
def plt_overhead(df, distance_metric):
    x_axis = get_xaxis_settings(df, distance_metric=distance_metric)
    y_axis = get_yaxis_settings(df, distance_metric=distance_metric)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x_axis['Distance']['col'],
        y=y_axis['Distance']['col'],
        line=dict(color=y_axis['Distance']['color'], width=1.2),
        hovertemplate=y_axis['Distance']['hovertemplate'],
        showlegend=False
    ))
    fig.update_layout(hovermode='x unified',  height=700, width=700, title='Overhead view of flight path', legend=dict(yanchor="top", y=1, xanchor="right", x=1))
    fig.update_xaxes(title=f"X-axis distance ({ x_axis['Distance']['metric'] })")
    fig.update_yaxes(title=f"Y-axis distance ({ y_axis['Distance']['metric'] })")
    return fig

plt_overhead(test_df, 'm')

In [65]:
def plt_3d_flight_path(df, distance_metric):
    x_axis = get_xaxis_settings(df, distance_metric=distance_metric)
    y_axis = get_yaxis_settings(df, distance_metric=distance_metric)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=x_axis['Distance']['col'], 
        y=y_axis['Distance']['col'], 
        z=y_axis['Elevation']['col'], 
        mode='lines',
        line=dict(
            color=y_axis['Elevation']['col'],
            colorscale='Turbo',
            width=3
        )
    ))
    #fig.add_trace(go.Surface(
    #    x=x_axis['Distance']['col'], 
    #    y=y_axis['Distance']['col'], 
    #    z=y_axis['Elevation']['col']
    #))
    fig.update_layout(
        title='Overhead view of flight path',
    )
    return fig

plt_3d_flight_path(test_df, 'm')

In [34]:
def debug(df, yaxis, speed_metric='km/u', distance_metric='m'):
    x_axis = get_xaxis_settings(df, distance_metric=distance_metric)
    y_axis = get_yaxis_settings(df, speed_metric=speed_metric)
    
    vert_speed_peaks = pu.indexes(y_axis[yaxis]['col'], thres=0.5, min_dist=1)
    vert_speed_lows = pu.indexes(-y_axis[yaxis]['col'], thres=0.5, min_dist=1)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x_axis['Horizontal Distance']['col'], 
        y=y_axis[yaxis]['col'], 
        name=yaxis,
        line=dict(color=y_axis[yaxis]['color'], width=1.2),
        hovertemplate=y_axis[yaxis]['hovertemplate'],
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=x_axis['Horizontal Distance']['col'][vert_speed_peaks], 
        y=y_axis[yaxis]['col'][vert_speed_peaks],
        mode='markers', 
        marker=dict(color='Green', size=8), 
        showlegend=False
    ))
    fig.add_trace(go.Scatter( 
        x=x_axis['Horizontal Distance']['col'][vert_speed_lows], 
        y=y_axis[yaxis]['col'][vert_speed_lows],
        mode='markers', 
        marker=dict(color='Red', size=8),
        showlegend=False
    ))
    fig.update_layout(hovermode="x unified", legend=dict(yanchor="top", y=1, xanchor="right", x=1))
    fig.update_xaxes(title=f"Horizontal distance ({ x_axis['Horizontal Distance']['metric'] })")
    fig.update_yaxes(title=f"{ yaxis } ({ y_axis[yaxis]['metric'] })")
    return fig
    
debug(test_df, 'Elevation', 'km/u', 'm')

In [ ]:
# Impossible!!!

def plt_touch_down():
    horz_speed_peaks = pu.indexes(landing_df.horz_speed_mph, thres=0.6, min_dist=1)
    horz_speed_lows = pu.indexes(-landing_df.horz_speed_mph, thres=0.5, min_dist=1)
    
    new_df = landing_df.iloc[horz_speed_peaks[-1]:get_stop()+1].reset_index(drop=True)
    s_speed_peaks = pu.indexes(np.diff(new_df.horz_speed_mph, 2), thres=0.1, min_dist=1)
    s_speed_lows = pu.indexes(-np.diff(new_df.horz_speed_mph, 2), thres=0.1, min_dist=1)
    
    speed = [new_df['horz_speed_mph'][s_speed_peaks[i+1]] - new_df['horz_speed_mph'][s_speed_peaks[i]] for i in range(0, len(s_speed_peaks)-1)]
    
    sorted_speed = sorted(range(len(speed)), key=lambda k: speed[k])
    print(sorted_speed)
    l = []
    for i in sorted_speed:
        if new_df['vert_speed_mph'][s_speed_peaks[i]] <= -1:
            l.append(i)
    print(l)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=new_df['horz_distance_m'], y=new_df['horz_speed_mph'], showlegend=False))
    fig.add_trace(go.Scatter(x=new_df['horz_distance_m'], y=new_df['vert_speed_mph'], showlegend=False))
    fig.add_trace(go.Scatter(mode='markers', x=new_df['horz_distance_m'][s_speed_peaks], y=new_df['horz_speed_mph'][s_speed_peaks],
        marker=dict(color='Green', size=6),showlegend=False
    ))
    fig.add_trace(go.Scatter(mode='markers', x=new_df['horz_distance_m'][s_speed_peaks], y=new_df['vert_speed_mph'][s_speed_peaks],
        marker=dict(color='Green', size=6),showlegend=False
    ))
    fig.add_trace(go.Scatter(mode='markers', x=[new_df['horz_distance_m'][s_speed_peaks[l[0]]]], y=[new_df['horz_speed_mph'][s_speed_peaks[l[0]]]],
        marker=dict(color='Blue', size=6),showlegend=False
    ))
    fig.show()
    
plt_touch_down()